In [ ]:
import json
with open("savings.json", 'r') as inputfile:
  try:
    base = json.load(inputfile)
  except:
    base = {}

In [ ]:
def getActualSchedule(dtstart, dtend):
  import urllib3
  http = urllib3.PoolManager()
  has_key = False
  for key in base.keys():
    if key == 'credentials':
      has_key = True
  if has_key:
    has_key = False
    for key in base['credentials'].keys():
      if key == 'authToken':
        has_key = True
  if not has_key or base['credentials']['authToken'] == "":
    r = http.request('POST', 'https://api.kadromierz.pl/security/authentication', fields={'email':base['credentials']['email'], 'password':base['credentials']['password']})
    base['credentials']['authToken'] = json.loads(r.data.decode('utf8').replace("'", '"'))['auth_token']
    print("Renewing auth token")
  http.headers['Authorization'] = 'AUTH-TOKEN token="' + base['credentials']['authToken'] + '"'
  start = str(dtstart.year) + '-' + str(dtstart.month) + '-' + str(dtstart.day)
  end = str(dtend.year) + '-' + str(dtend.month) + '-' + str(dtend.day)
  r = http.request('GET', 'https://api.kadromierz.pl/locations/7738/schedule?from=' + start + '&to=' + end + '&show_drafts=false')
  if r.status != 200 and r.status != 404:
    base['credentials']['authToken'] = ""
  return json.loads(r.data.decode('utf8').replace("'", '"'))

In [ ]:
def getCalendarForEmp(empID: int):
  from datetime import datetime, timedelta
  s = datetime.today()
  e = s + timedelta(7)
  schedule = getActualSchedule(s, e)
  emp = None
  for e in schedule['schedule']['employees']:
    if int(e['id']) == empID:
      emp = e
      break
  if emp == None:
    print("No employeee")
    return 0
  calendar = []
  for shift in emp['shifts_for_other_locations']:
    start = datetime(int(shift['start_timestamp'][0:4]), int(shift['start_timestamp'][5:7]), int(shift['start_timestamp'][8:10]), int(shift['start_timestamp'][11:13]), int(shift['start_timestamp'][14:16]), int(shift['start_timestamp'][17:19]))
    end = datetime(int(shift['end_timestamp'][0:4]), int(shift['end_timestamp'][5:7]), int(shift['end_timestamp'][8:10]), int(shift['end_timestamp'][11:13]), int(shift['end_timestamp'][14:16]), int(shift['end_timestamp'][17:19]))
    new_event = {'dtstart' : start, 'dtend' : end, 'location' : shift['job_title']['title']}
    calendar.append(new_event)
  return calendar